In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import torch

In [ ]:
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = False
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)
plt.rc('axes', labelsize=12)
mpl.rcParams['figure.dpi'] = 300

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# Import the various experiments we need for the notebook
from sva.experiments import Simple2d

# Import the helper functions for Gaussian Processes
from sva.models.gp import EasySingleTaskGP

# A simple 2d example

In [ ]:
experiment = Simple2d()
experiment.initialize_data(n=35, protocol="random", seed=123)
extent = experiment.get_experimental_domain_mpl_extent()
X = experiment.data.X

In [ ]:
# Get the ground truth results
x = experiment.get_dense_coordinates(ppd=100)
y = experiment(x)

In [ ]:
# Train a GP on the current data
gp = EasySingleTaskGP.from_default(X, experiment(X))
gp.fit_mll()
mu, var = gp.predict(x)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(6, 3), sharex=True, sharey=True)

ax = axs[0]
ax.imshow(y.reshape(100, 100).T, extent=extent, interpolation='nearest', origin='lower')
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5)
ax.set_title("Ground Truth")


ax = axs[1]
ax.imshow(mu.reshape(100, 100).T, extent=extent, interpolation='nearest', origin='lower')
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5)
ax.set_title("GP Prediction")


ax = axs[2]
ax.imshow(mu.reshape(100, 100).T - y.reshape(100, 100).T, extent=extent, interpolation='nearest', origin='lower', cmap="RdBu")
ax.scatter(X[:, 0], X[:, 1], color="black", s=0.5)
ax.set_title("Delta")

plt.show()